<a href="https://colab.research.google.com/github/Iambinhh/Transplant_Patient/blob/main/Transplant_Patients_Fuzzy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install scikit-fuzzy


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 994.0/994.0 KB 11.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-fuzzy: filename=scikit_fuzzy-0.4.2-py3-none-any.whl size=894088 sha256=5e7eb74722fc8212e257267246d476aa2844b3efe5d77ffd104ef57de3a5056c
  Stored in directory: /root/.cache/pip/wheels/2c/04/80/7eefb1a2de7d36aefd06432fab2a1486caf0a0596a7067391a
Successfully built scikit-fuzzy


In [ ]:
#ProtFIS
from skfuzzy.control.controlsystem import Antecedent
import numpy as np
import  skfuzzy as fuzz
from skfuzzy import control as ctrl

# Với biến đầu ra là proteinuria chú ý sử dụng: Consequent

glycaemia   = ctrl.Antecedent(np.arange(0,151,1),'glycaemia')
sirolimus   = ctrl.Antecedent(np.arange(0,21,1),'sirolimus')

proteinuria = ctrl.Consequent(np.arange(0,3501,1),'proteinuria')#the output result

# Tạo hàm liên thuộc
# Lấy biến glycaemia với normal --> good, impaired --> alarm,  diabetes --> danger với hàm trimf (universe) hàm phổ quát lấy giá trị mảng food để truyền vào
glycaemia['good'] = fuzz.trapmf(glycaemia.universe,[0,70,80,100])
glycaemia['alarm'] = fuzz.trimf(glycaemia.universe,[101,105,125])
glycaemia['danger'] = fuzz.trapmf(glycaemia.universe,[125,135,150,150])

# Lấy biến sirolimus với alarm down , sufficient --> suff,  good, alarm up, danger với hàm trimf (universe) hàm phổ quát lấy giá trị mảng food để truyền vào
sirolimus['alarm down'] = fuzz.trapmf(sirolimus.universe,[0, 0, 3, 5])
sirolimus['suff'] = fuzz.trimf(sirolimus.universe,[4, 5, 7])
sirolimus['good'] = fuzz.trimf(sirolimus.universe,[6, 9, 12])
sirolimus['alarm up'] = fuzz.trimf(sirolimus.universe,[11, 12 ,14])
sirolimus['danger'] = fuzz.trapmf(sirolimus.universe,[13, 16, 20, 20])

# Lấy biến proteinuria với physiological --> alarm , light, moderate,  severe --> suff, nephrotic --> danger với hàm trimf (universe) hàm phổ quát lấy giá trị mảng food để truyền vào
# Đổi đơn vị mg sang g
proteinuria['good'] = fuzz.trapmf(proteinuria.universe,[0, 0, 125, 150])
proteinuria['suff'] = fuzz.trimf(proteinuria.universe,[150, 350, 500])
proteinuria['alterate'] = fuzz.trimf(proteinuria.universe,[500, 800, 1000])
proteinuria['alarm'] = fuzz.trimf(proteinuria.universe,[1000, 2100, 3000])
proteinuria['danger'] = fuzz.trapmf(proteinuria.universe,[2800, 3000, 3500, 3500])

glycaemia.view()
sirolimus.view()
proteinuria.view()

# Tạo luật
rule1 = ctrl.Rule(glycaemia['good'] & sirolimus['good'] , proteinuria['good'])
rule2 = ctrl.Rule(glycaemia['danger'] & sirolimus['danger'] , proteinuria['danger'])
rule3 = ctrl.Rule(glycaemia['good'] & sirolimus['suff'] , proteinuria['suff'])
rule4 = ctrl.Rule(glycaemia['alarm'] & sirolimus['alarm up'] , proteinuria['alarm'])
rule5 = ctrl.Rule(glycaemia['alarm'] & sirolimus['suff'] , proteinuria['alterate'])
rule6 = ctrl.Rule(glycaemia['good'] & sirolimus['alarm up'] , proteinuria['alterate'])
rule7 = ctrl.Rule(glycaemia['alarm'] & sirolimus['good'] , proteinuria['suff'])
rule8 = ctrl.Rule(glycaemia['good'] & sirolimus['alarm down'] , proteinuria['suff'])

rule9 = ctrl.Rule(glycaemia['alarm'] & sirolimus['alarm down'] , proteinuria['suff'])
rule10 = ctrl.Rule(glycaemia['danger'] & sirolimus['alarm down'] , proteinuria['alterate'])
rule11 = ctrl.Rule(glycaemia['danger'] & sirolimus['suff'] , proteinuria['alarm'])
rule12= ctrl.Rule(glycaemia['danger'] & sirolimus['good'] , proteinuria['suff'])
rule13= ctrl.Rule(glycaemia['danger'] & sirolimus['alarm up'] , proteinuria['danger'])
rule14= ctrl.Rule(glycaemia['good'] & sirolimus['danger'] , proteinuria['suff'])
rule15= ctrl.Rule(glycaemia['alarm'] & sirolimus['danger'] , proteinuria['alarm'])
# Thu thập dữ liệu ControlSystem
# Học dữ liệu SystemSimulation --> Truyền giá trị (đọc từ cảm biến)(thường đưa vào vòng lặp để quét)-->compute để tính toán
tipping_ctrl = ctrl.ControlSystem([rule1, rule2, rule3, rule4, rule5, rule6, rule7, rule8,rule9,rule10,rule11,rule12,rule13,rule14,rule15])
tipping = ctrl.ControlSystemSimulation(tipping_ctrl)

gly_input=float(input("Enter the glycaemia:"))
while gly_input>150 or gly_input<0:
  try:
   gly_input= float(input("Please re-enter the value: "))
  except ValueError:
    print("Not in range or wrongd datatype")

blood_level_input=float(input("Enter your blood-level: "))
while blood_level_input>21 or blood_level_input<0:
  try:
    blood_level_input= float(input("Please re-enter the value: "))
  except ValueError:
    print("Not in range or wrong datatype")

tipping.input['glycaemia'] = gly_input
tipping.input['sirolimus'] = blood_level_input
tipping.compute()
proteinuria.view(sim = tipping)

#printing_out the values
print("The protenuria is between in 150 and 3000")
print(tipping.output["proteinuria"])
if (tipping.output["proteinuria"]<150):
  print("Physiological protein")
if (tipping.output["proteinuria"]>150 and tipping.output["proteinuria"]<500):
  print("Light proteinuria")
if (tipping.output["proteinuria"]>500 and tipping.output["proteinuria"]<1000):
  print("Moderate proteinuria")
if (tipping.output["proteinuria"]>1000 and tipping.output["proteinuria"]<3000):
  print("Serve proteinuria")
else:
  print("Nephrotic proteinuria")
